In [1]:
import cv2
import numpy as np
from dataset import load_hoda
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D
from keras.models import Sequential
import matplotlib.pyplot as plt

2024-07-17 03:19:46.547090: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-17 03:19:46.571482: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 03:19:46.571508: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 03:19:46.572104: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-17 03:19:46.576244: I tensorflow/core/platform/cpu_feature_guar

In [2]:
tf.config.experimental.list_physical_devices()

2024-07-17 03:19:48.779838: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [3]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.1


In [4]:
x_train, y_train, x_test, y_test = load_hoda(training_sample_size=50000, test_sample_size=10000, size=30)

In [5]:
x_train = x_train.reshape((-1, 30, 30, 1)) / 255.0
x_test = x_test.reshape((-1, 30, 30, 1)) / 255.0

In [6]:
model = Sequential([
    Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(30, 30, 1)),
    MaxPooling2D(),
    Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dropout(0.4),
    Dense(units=10, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 15, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 128)         0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dropout (Dropout)           (None, 6272)              0

In [7]:
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [8]:
with tf.device('gpu:0'):
    model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 18s 28ms/step - loss: 0.1423 - accuracy: 0.9582 - val_loss: 0.0527 - val_accuracy: 0.9855
Epoch 2/10
625/625 [==============================] - 19s 30ms/step - loss: 0.0445 - accuracy: 0.9868 - val_loss: 0.0354 - val_accuracy: 0.9907
Epoch 3/10
625/625 [==============================] - 19s 31ms/step - loss: 0.0308 - accuracy: 0.9912 - val_loss: 0.0295 - val_accuracy: 0.9921
Epoch 4/10
625/625 [==============================] - 19s 31ms/step - loss: 0.0267 - accuracy: 0.9920 - val_loss: 0.0302 - val_accuracy: 0.9911
Epoch 5/10
625/625 [==============================] - 19s 31ms/step - loss: 0.0203 - accuracy: 0.9938 - val_loss: 0.0308 - val_accuracy: 0.9905
Epoch 6/10
625/625 [==============================] - 19s 31ms/step - loss: 0.0170 - accuracy: 0.9945 - val_loss: 0.0277 - val_accuracy: 0.9923
Epoch 7/10
625/625 [==============================] - 20s 31ms/step - loss: 0.0146 - accuracy: 0.9955 - val_loss: 0.0275 - val_accuracy:

In [10]:
model.save('model.h5')